# Data Acquisition. 0 Data Structure of Files

In [1]:
# installations (be careful!!!)
#!pip install missingno

### THIS WAS ALREADY HERE

In [239]:
# all modules
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import requests
import re
import datetime
import numpy as np
from sklearn.preprocessing import OneHotEncoder

from zipfile import ZipFile

In [235]:
# global stuff

warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_rows', 13)
pd.set_option('display.max_columns', 13)

In [4]:
# load sql extension for jupyter
%load_ext sql
%config SqlMagic.autocommit = False

In [5]:
# load database
%sql sqlite:///../data/raw/raw_data_project_m1.db

In [6]:
%%sql tables_structure <<

select * 
from 
    sqlite_master

where 
    type='table' AND
    name NOT LIKE 'sqlite_%'


 * sqlite:///../data/raw/raw_data_project_m1.db
Done.
Returning data to local variable tables_structure


In [36]:
#Show whats in the DB
tables_structure.DataFrame()

## COMENTARIOS DE EXPLORACIÓN DBeaver:
##    - No están conectactas pero tienen en común el campo uuid que es un hash

,type,name,tbl_name,rootpage,sql
0,table,personal_info,personal_info,2,CREATE TABLE personal_info (\n\tuuid TEXT NOT ...
1,table,country_info,country_info,401,CREATE TABLE country_info (\n\tuuid TEXT NOT N...
2,table,career_info,career_info,762,CREATE TABLE career_info (\n\tuuid TEXT NOT NU...
3,table,poll_info,poll_info,1168,CREATE TABLE poll_info (\n\tuuid TEXT NOT NULL...


## 1_ DF PESONAL_INFO 

In [353]:
%%sql personal_info <<

select * from personal_info;

 * sqlite:///../data/raw/raw_data_project_m1.db
Done.
Returning data to local variable personal_info


In [354]:
df_personal_info = personal_info.DataFrame()

### Información descriptiva de la tabla

Contiene 5 columnas con 9649 rows... todas son de tipo object (esto se podría optimizar) y tienen poca variación de valores

In [355]:
df_personal_info.shape 
# df_personal_info.info() --> todo objects

(9649, 5)

In [356]:
df_personal_info.describe().T

,count,unique,top,freq
uuid,9649,9649,fa4adae0-dc1f-0133-dc65-0a81e8b09a82,1
age,9649,52,40 years old,477
gender,9649,5,male,3796
dem_has_children,9649,4,yES,2561
age_group,9649,4,40_65,4896


¿Cómo son los valores dentro de cada columna?

### Funciones y Resultado

In [357]:
def ageStr_to_ageNum(serie):
    """
    INPUT -> serie df[] = ['61 years old', '57 years old', '32 years old'] -> full strings
    OUPUT -> serie df[] = [ 61 57 32 45 41 1990 2000]                      -> only integers
    """
    return serie.apply(lambda x: re.sub('\D', '', x)).astype(int)


def year_to_age(serie):
    """
    INPUT -> serie:  df[] = [ 61 57 32 45 41 1990 2000]   -> ages + years
    OUTPUT -> serie: df[] = [ 61 57 32 45 41   30   20]   -> only ages
    """   
    year_now = datetime.datetime.today().year
    return serie.apply(lambda x: year_now - x if int(x) > 200 else x)

# Funciones para homogeneizar genders

def gender_homogenization_Fem(serie):
    """
    INPUT  -> female, FeMale, Fem
    OUTPUT -> F
    """
    return serie.apply(lambda x: re.sub('^female|^Fem|^FeMale', 'F', x))

def gender_homogenization_Masc(serie):
    """
    INPUT  -> male, Male
    OUTPUT -> M
    """
    return serie.apply(lambda x: re.sub('^male|^Male', 'M', x))

# Funciones para yes or no questions

def yes_no_to_bool(serie):
    """
    INPUT  -> YES yes Yes yES No NO nO no  -> type str
    OUTPUT ->   1   1   1   1  0  0  0  0  -> type bool
    
    """
    serie = serie.apply(lambda x: re.sub('^y\w+|^Y\w+', '1', x))
    serie = serie.apply(lambda x: re.sub('^n\w+|^N\w+', '0', x)).astype(int)
    return serie.astype(bool)

# Dividir cols en booleanos

def separate_df_to_bools(df, cols_to_separate, cols_separated):
    df_encoder = OneHotEncoder(dtype=bool, sparse=True)

    df = pd.DataFrame(df_encoder.fit_transform(df[cols_to_separate]).toarray(),columns=cols_separated)
    
    return df

In [358]:
df_personal_info['age'] = ageStr_to_ageNum(df_personal_info['age'])
df_personal_info['age'] = year_to_age(df_personal_info['age'])

df_personal_info['gender'] = gender_homogenization_Fem(df_personal_info['gender'])
df_personal_info['gender'] = gender_homogenization_Masc(df_personal_info['gender'])

df_personal_info['dem_has_children'] = yes_no_to_bool(df_personal_info['dem_has_children'])

In [359]:
### Se definen las columnas a añadir como booleanos
initial_cols = ['gender', 'age_group']
final_cols = ['gender_Female', 'gender_Male', 'ageGroup_14_25', 'ageGroup_26_39', 'ageGroup_40_65', 'ageGroup_juvenile']

# las columnas a separar siempre en ORDEN ALFABÉTICO

new_bool_df = separate_df_to_bools(df_personal_info, initial_cols, final_cols)

In [360]:
df_personal_info = df_personal_info.join(other= new_bool_df, on=None, how='left', sort=False)

In [361]:
df_personal_info.head(10)

,uuid,age,gender,dem_has_children,age_group,gender_Female,gender_Male,ageGroup_14_25,ageGroup_26_39,ageGroup_40_65,ageGroup_juvenile
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,61,M,False,40_65,False,True,False,False,True,False
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,57,M,True,40_65,False,True,False,False,True,False
2,83127080-da3d-0133-c74f-0a81e8b09a82,32,M,False,26_39,False,True,False,True,False,False
3,15626d40-db13-0133-ea5c-0a81e8b09a82,45,M,True,40_65,False,True,False,False,True,False
4,24954a70-db98-0133-4a64-0a81e8b09a82,41,F,True,40_65,True,False,False,False,True,False
5,28583060-dbe2-0133-ff9c-0a81e8b09a82,30,F,False,26_39,True,False,False,True,False,False
6,b50dbb80-da53-0133-8956-0a81e8b09a82,30,F,False,26_39,True,False,False,True,False,False
7,dd743bb0-d9a5-0133-53b2-0a81e8b09a82,47,M,False,40_65,False,True,False,False,True,False
8,4fde2370-da48-0133-3a42-0a81e8b09a82,34,F,False,26_39,True,False,False,True,False,False
9,9949c4c0-da5f-0133-c832-0a81e8b09a82,26,F,True,juvenile,True,False,False,False,False,True


In [362]:
df_personal_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9649 entries, 0 to 9648
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   uuid               9649 non-null   object
 1   age                9649 non-null   int64 
 2   gender             9649 non-null   object
 3   dem_has_children   9649 non-null   bool  
 4   age_group          9649 non-null   object
 5   gender_Female      9649 non-null   bool  
 6   gender_Male        9649 non-null   bool  
 7   ageGroup_14_25     9649 non-null   bool  
 8   ageGroup_26_39     9649 non-null   bool  
 9   ageGroup_40_65     9649 non-null   bool  
 10  ageGroup_juvenile  9649 non-null   bool  
dtypes: bool(7), int64(1), object(3)
memory usage: 367.6+ KB


## 2_ DF COUNTRY_INFO 

In [18]:
%%sql country_info <<

select * from country_info;

 * sqlite:///../data/raw/raw_data_project_m1.db
Done.
Returning data to local variable country_info


In [19]:
df_country_info = country_info.DataFrame()
df_country_info.head()

,uuid,country_code,rural
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,AT,countryside
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,AT,urban
2,83127080-da3d-0133-c74f-0a81e8b09a82,AT,city
3,15626d40-db13-0133-ea5c-0a81e8b09a82,AT,Country
4,24954a70-db98-0133-4a64-0a81e8b09a82,AT,city


In [20]:
df_country_info.shape

(9649, 3)

In [21]:
df_country_info.info() # Todo son objects

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9649 entries, 0 to 9648
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uuid          9649 non-null   object
 1   country_code  9649 non-null   object
 2   rural         9649 non-null   object
dtypes: object(3)
memory usage: 226.3+ KB


In [22]:
df_country_info.describe().T

,count,unique,top,freq
uuid,9649,9649,44628ae0-dcb3-0133-316d-0a81e8b09a82,1
country_code,9649,28,DE,1420
rural,9649,6,urban,3842


In [23]:
for item in df_country_info['country_code'].unique():
    print(f'{item}\t')

AT	
BE	
BG	
CY	
CZ	
DE	
DK	
EE	
ES	
FI	
FR	
GB	
GR	
HR	
HU	
IE	
IT	
LT	
LU	
LV	
MT	
NL	
PL	
PT	
RO	
SE	
SI	
SK	


In [24]:
df_country_info['rural'].value_counts()

urban          3842
city           1664
rural          1517
Non-Rural      1372
Country         772
countryside     482
Name: rural, dtype: int64

## 3_ DF CAREER_INFO 

In [25]:
%%sql career_info <<

select * from career_info;

 * sqlite:///../data/raw/raw_data_project_m1.db
Done.
Returning data to local variable career_info


In [26]:
df_career_info = career_info.DataFrame()
df_career_info.head()

,uuid,dem_education_level,dem_full_time_job,normalized_job_code
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,no,no,None
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0
2,83127080-da3d-0133-c74f-0a81e8b09a82,None,no,None
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f


In [27]:
df_career_info.shape

(9649, 4)

In [28]:
df_career_info.info() # todo son objects

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9649 entries, 0 to 9648
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   uuid                 9649 non-null   object
 1   dem_education_level  8986 non-null   object
 2   dem_full_time_job    9649 non-null   object
 3   normalized_job_code  5702 non-null   object
dtypes: object(4)
memory usage: 301.7+ KB


In [29]:
df_career_info.describe().T

,count,unique,top,freq
uuid,9649,9649,44628ae0-dcb3-0133-316d-0a81e8b09a82,1
dem_education_level,8986,4,medium,3578
dem_full_time_job,9649,2,yes,5702
normalized_job_code,5702,156,775190277a849cba701b306a7b374c0a,101


In [30]:
df_career_info['dem_education_level'].value_counts()

medium    3578
high      3270
low       1815
no         323
Name: dem_education_level, dtype: int64

In [31]:
df_career_info['dem_full_time_job'].value_counts()

yes    5702
no     3947
Name: dem_full_time_job, dtype: int64

In [32]:
df_career_info['normalized_job_code'].value_counts()

775190277a849cba701b306a7b374c0a    101
fcaf4aac737e53ad24b9f32f08b8740f     88
d4624ed8068d24948e18303212593cd6     88
049a3f3a2b5f85cb2971ba77ad66e10c     88
dac2e921c1f887d9efc4c06fa33c1397     88
                                   ... 
58eaf730050d2e23c7361fbb782e30a2      6
7aecc34a90868aac810bf3693b11534e      5
5285758deb04c98f58cbae2747195f33      5
631bc3a12ed11340fc94ebfffa8f5f4e      4
b42566382f2d3deeca56d3667004fc9e      4
Name: normalized_job_code, Length: 156, dtype: int64

## 4_ DF POLL_INFO 

In [33]:
%%sql poll_info <<

select * from poll_info;

 * sqlite:///../data/raw/raw_data_project_m1.db
Done.
Returning data to local variable poll_info


In [34]:
df_poll_info = poll_info.DataFrame()
df_poll_info.head()

,uuid,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,f6e7ee00-deac-0133-4de8-0a81e8b09a82,I know something about it,I would not vote,None of the above,None of the above,None of the above
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...


In [35]:
df_poll_info.shape

(9649, 6)

In [36]:
df_poll_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9649 entries, 0 to 9648
Data columns (total 6 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   uuid                                                 9649 non-null   object
 1   question_bbi_2016wave4_basicincome_awareness         9649 non-null   object
 2   question_bbi_2016wave4_basicincome_vote              9649 non-null   object
 3   question_bbi_2016wave4_basicincome_effect            9649 non-null   object
 4   question_bbi_2016wave4_basicincome_argumentsfor      9649 non-null   object
 5   question_bbi_2016wave4_basicincome_argumentsagainst  9649 non-null   object
dtypes: object(6)
memory usage: 452.4+ KB


In [37]:
df_poll_info.describe().T

,count,unique,top,freq
uuid,9649,9649,44628ae0-dcb3-0133-316d-0a81e8b09a82,1
question_bbi_2016wave4_basicincome_awareness,9649,4,I know something about it,3389
question_bbi_2016wave4_basicincome_vote,9649,5,I would probably vote for it,3272
question_bbi_2016wave4_basicincome_effect,9649,9,A basic income would not affect my work choices,3349
question_bbi_2016wave4_basicincome_argumentsfor,9649,674,None of the above,2163
question_bbi_2016wave4_basicincome_argumentsagainst,9649,881,None of the above,1381


In [38]:
df_poll_info['question_bbi_2016wave4_basicincome_awareness'].value_counts()

I know something about it              3389
I understand it fully                  2389
I have heard just a little about it    2305
I have never heard of it               1566
Name: question_bbi_2016wave4_basicincome_awareness, dtype: int64

In [39]:
df_poll_info['question_bbi_2016wave4_basicincome_vote'].value_counts()

I would probably vote for it        3272
I would vote for it                 2819
I would probably vote against it    1439
I would not vote                    1159
I would vote against it              960
Name: question_bbi_2016wave4_basicincome_vote, dtype: int64

In [40]:
df_poll_info['question_bbi_2016wave4_basicincome_effect'].value_counts()

A basic income would not affect my work choices    3349
‰Û_ spend more time with my family                 1387
None of the above                                  1227
‰Û_ gain additional skills                          970
‰Û_ look for a different job                        640
‰Û_ work less                                       629
‰Û_ do more volunteering work                       615
‰Û_ work as a freelancer                            465
‰Û_ stop working                                    367
Name: question_bbi_2016wave4_basicincome_effect, dtype: int64

In [41]:
for item in df_poll_info['question_bbi_2016wave4_basicincome_argumentsfor'].unique():
    print(f'{item}\n')
    
"""Es una pregunta donde se puede elegir un núm de posibilidades"""

None of the above

It increases appreciation for household work and volunteering | It encourages financial independence and self-responsibility | It reduces anxiety about financing basic needs

It creates more equality of opportunity

It reduces anxiety about financing basic needs

It encourages financial independence and self-responsibility | It reduces anxiety about financing basic needs

It increases appreciation for household work and volunteering | It reduces anxiety about financing basic needs

It creates more equality of opportunity | It reduces anxiety about financing basic needs | It reduces bureaucracy and administrative expenses

It increases solidarity, because it is funded by everyone

It increases appreciation for household work and volunteering

It reduces anxiety about financing basic needs | It creates more equality of opportunity | It encourages financial independence and self-responsibility | It increases appreciation for household work and volunteering

It encourage


It reduces anxiety about financing basic needs | It reduces bureaucracy and administrative expenses | It increases appreciation for household work and volunteering

It creates more equality of opportunity | It increases appreciation for household work and volunteering | It encourages financial independence and self-responsibility

It increases solidarity, because it is funded by everyone | It creates more equality of opportunity | It reduces anxiety about financing basic needs | It encourages financial independence and self-responsibility | It increases appreciation for household work and volunteering

It reduces bureaucracy and administrative expenses | It encourages financial independence and self-responsibility | It reduces anxiety about financing basic needs

It reduces bureaucracy and administrative expenses | It increases appreciation for household work and volunteering | It reduces anxiety about financing basic needs | It creates more equality of opportunity | It encourages fin

It creates more equality of opportunity | It increases appreciation for household work and volunteering | It encourages financial independence and self-responsibility | It reduces anxiety about financing basic needs | It increases solidarity, because it is funded by everyone

It reduces anxiety about financing basic needs | It reduces bureaucracy and administrative expenses | It encourages financial independence and self-responsibility | It increases appreciation for household work and volunteering

It creates more equality of opportunity | It encourages financial independence and self-responsibility | It increases solidarity, because it is funded by everyone | It reduces bureaucracy and administrative expenses | It increases appreciation for household work and volunteering

It reduces anxiety about financing basic needs | It increases solidarity, because it is funded by everyone | It encourages financial independence and self-responsibility | It increases appreciation for household wo

'Es una pregunta donde se puede elegir un núm de posibilidades'

In [42]:
for item in df_poll_info['question_bbi_2016wave4_basicincome_argumentsagainst'].unique():
    print(f'{item}\n')

None of the above

It might encourage people to stop working

Foreigners might come to my country and take advantage of the benefit

It is impossible to finance | It might encourage people to stop working | It increases dependence on the state

It is impossible to finance | It might encourage people to stop working | It is against the principle of linking merit and reward

Foreigners might come to my country and take advantage of the benefit | It might encourage people to stop working | It increases dependence on the state

It might encourage people to stop working | Foreigners might come to my country and take advantage of the benefit | It is impossible to finance | Only the people who need it most should get something from the state

Foreigners might come to my country and take advantage of the benefit | It is impossible to finance

It is against the principle of linking merit and reward | It might encourage people to stop working | It increases dependence on the state

Foreigners mi

Las cols anteriores se deberían separar en opciones y hacer una matriz de True o False, de esta forma se multiplicidad de opciones tendría sentido

## CONEXIÓN API

In [44]:
requests.get('http://dataatwork.org/data/')

<Response [200]>